In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import distributions
from torch.nn.parameter import Parameter
import torch.utils.data as data_utils
from core import * 
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 7.0)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'core'

### Please download data 
https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data
and put it in the same directory than this notebook

In [24]:
def read_arrhythmia(shuffle=False, dropmissing=False):
    data = pd.read_csv(
    "arrhythmia.data",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?", header=None)
    if dropmissing: 
        data.dropna()
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    nTrain = data.shape[0] - data.shape[0]//10
    data.rename(columns={1:'Sex', 279: 'Target'},inplace=True)
    data['Target'] = (data['Target']>1)*1
    data = pd.get_dummies(data, columns=['Sex'])
    to_protect = data['Sex_1'].values
    label = data['Target'].values
    cols = data.columns.tolist()
    n = int(cols.index('Sex_0'))
    cols = cols[n:n+2] + cols[:n] + cols[n+2:]  
    data = data[cols]
    normalized = (data-data.mean())/data.std()
    for n in ['Sex_0', 'Sex_1', 'Target']:
        normalized[n] = data[n]
    data = normalized
    data.fillna(0,inplace=True)
    #data = data.drop('Sex_0', 1)
    #data = data.drop('Sex_1', 1)
    #data = data.drop('Target', 1)
    return data, to_protect, label#data.iloc[:nTrain, :], to_protect[:nTrain], data.iloc[nTrain:, :], to_protect[nTrain:]

#encoded_data, to_protect, encoded_data_test, to_protect_test = read_arrhythmia()
data, to_protect, label = read_arrhythmia()
#data.head() 
#np.shape(label)
#data.to_csv('arrhythmia_X.csv', sep='\t')
data.head()

,Sex_0,Sex_1,0,2,3,4,5,6,7,8,...,270,271,272,273,274,275,276,277,278,Target
0,1,0,1.732520,0.640617,0.713024,0.135355,0.844010,0.113584,0.113683,1.200140,...,0.50828,-0.013824,0.278312,-0.079458,0.0,1.108325,1.176433,0.294277,1.077476,1
1,0,1,0.578671,-0.031962,-0.251365,-0.515501,0.420303,1.012179,-0.587912,-1.974876,...,0.50828,-0.157797,0.727766,-0.079458,0.0,-0.905886,0.615444,0.079525,0.504315,1
2,1,0,0.457213,0.156360,1.617140,3.194376,0.174999,0.562881,0.422385,0.464465,...,0.50828,0.130149,-0.470778,-0.079458,0.0,-0.618141,1.527052,-0.520301,1.055848,1
3,1,0,0.517942,0.237069,1.556865,0.721125,1.044714,0.383162,0.254002,2.051973,...,0.50828,0.907606,-0.370899,-0.079458,0.0,-0.330397,0.966062,1.131071,1.737152,0
4,1,0,1.732520,0.640617,0.713024,-0.059902,0.576406,-0.215902,0.197875,0.503185,...,0.50828,1.166758,-1.070050,-0.079458,0.0,-1.769119,1.877670,0.449787,1.802039,1


In [22]:
Y = data.Target #pd.DataFrame(data=data,columns=['Target']) 
X = data #pd.DataFrame(data=data[0:,0:],columns=feature_names) #X[0,1:] #index=X[0:,0]
A = X.Sex_1
X = X.drop('Sex_0', 1)
X = X.drop('Sex_1', 1)
X = X.drop('Target', 1)
#X = pd.DataFrame(list(X.items()),columns=feature_names)
X.to_csv('arrhythmia_X.csv', sep='\t')
A.to_csv('arrhythmia_A.csv', sep='\t')
Y.to_csv('arrhythmia_Y.csv', sep='\t')

/Users/omid/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':
/Users/omid/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # Remove the CWD from sys.path while we load stuff.


In [22]:
# Hyper Parameters 
input_size = encoded_data.shape[1]-1
num_classes = 2
num_epochs = 40
batch_size = 32
batchRenyi = 128.
learning_rate = 3e-4
lambda_renyi = 8. *  4. * batchRenyi/batch_size

class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

class NetRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetRegression, self).__init__()
        size = 80
        self.first = nn.Linear(input_size, size)
        self.fc = nn.Linear(size, size-20)
        self.fc2 = nn.Linear(size-20, size)
        self.last = nn.Linear(size, num_classes)
        self.d = nn.Dropout(0.5)
         
    
    def forward(self, x):
        out = F.selu( self.first(x) )
        #out = self.d(out)
        #out = F.selu( self.fc(out) )
        #out = F.selu( self.fc2(out) )    
        out = self.last(out)
        return out
    
    
    
cfg_factory=namedtuple('Config', 'model loader batch_size num_epochs lambda_renyi batchRenyi learning_rate input_size num_classes' )
config = cfg_factory(NetRegression, read_arrhythmia, batch_size, num_epochs, lambda_renyi, batchRenyi, learning_rate, input_size, num_classes)

In [23]:
#Acc 0.77
results_1 = pd.DataFrame([go(config,deleteSensitiveInfo=True, reg=False) for _ in range(10)])
results_1.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.733333,0.727822,0.176069,0.189766,0.188727,0.159443,0.187221,0.159453,0.181431,0.213220,...,0.716883,0.697272,0.673574,0.682791,0.718620,0.702380,0.676786,0.668875,0.637943,0.119228
std,0.083148,0.084682,0.169823,0.177700,0.169592,0.151005,0.135497,0.111925,0.105025,0.089484,...,0.213336,0.210961,0.189144,0.192589,0.244987,0.216799,0.212041,0.178573,0.163953,0.071011
min,0.577778,0.580040,0.011111,0.011111,0.011111,0.011111,0.064935,0.022222,0.022222,0.038462,...,0.435606,0.435606,0.348485,0.380165,0.380165,0.418182,0.313636,0.313636,0.450929,0.018221
25%,0.672222,0.670640,0.051340,0.085633,0.091259,0.071023,0.089692,0.075624,0.151515,0.182517,...,0.603196,0.562843,0.554536,0.533967,0.554218,0.547894,0.536532,0.565579,0.512682,0.069007
50%,0.777778,0.760776,0.111089,0.105245,0.134343,0.109091,0.128399,0.161616,0.190522,0.211039,...,0.637681,0.650746,0.733201,0.698973,0.677443,0.646791,0.639191,0.681893,0.560023,0.112112
75%,0.777778,0.783834,0.262581,0.256469,0.194730,0.191123,0.222378,0.209821,0.212662,0.231061,...,0.832939,0.816913,0.781101,0.840722,0.848168,0.848168,0.853715,0.770022,0.803054,0.174136
max,0.822222,0.839069,0.500000,0.571429,0.571429,0.446429,0.446429,0.400000,0.400000,0.400000,...,1.140496,1.150000,0.940909,0.940909,1.176136,1.045455,1.011494,0.919540,0.886195,0.225203


In [24]:
#Acc 0.75±0.05 DEO 0.05±0.02
results_2 = pd.DataFrame([go(config,deleteSensitiveInfo=True, floor=True, fdiv=binary_binary_renyi2_differentiable) for _ in range(10)])
results_2.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.773333,0.768178,0.103893,0.105282,0.105282,0.117782,0.143059,0.148831,0.156441,0.152870,...,0.897141,0.881557,0.819785,0.887059,0.892607,0.919026,0.994705,1.150526,0.603425,0.097421
std,0.089993,0.089932,0.076014,0.076847,0.076847,0.093740,0.091602,0.068460,0.112287,0.109802,...,0.312328,0.361158,0.335459,0.457269,0.417422,0.367627,0.529841,0.937921,0.164549,0.057688
min,0.577778,0.570000,0.022727,0.022727,0.022727,0.022727,0.022727,0.079365,0.000000,0.000000,...,0.608974,0.531469,0.439560,0.398601,0.265734,0.292308,0.292308,0.292308,0.355811,0.032035
25%,0.755556,0.748452,0.047917,0.047917,0.047917,0.047917,0.081495,0.087500,0.087500,0.095238,...,0.658854,0.591346,0.596591,0.615625,0.629514,0.688988,0.652282,0.723214,0.504442,0.072258
50%,0.777778,0.777976,0.090441,0.090441,0.090441,0.090441,0.138889,0.140025,0.162418,0.123529,...,0.776042,0.708333,0.687500,0.661932,0.815476,0.841146,0.854167,0.854167,0.590197,0.086908
75%,0.838889,0.826943,0.129167,0.139583,0.139583,0.170833,0.198611,0.194643,0.194643,0.194444,...,1.044444,1.204444,1.132937,1.211538,1.134066,1.192308,1.305769,1.269231,0.654204,0.101163
max,0.888889,0.883333,0.261905,0.261905,0.261905,0.277778,0.277778,0.277778,0.402778,0.402778,...,1.461538,1.461538,1.315385,1.619048,1.619048,1.416667,2.125000,3.642857,0.958318,0.246437


In [25]:
#Acc 0.75±0.05 DEO 0.05±0.02
results_3 = pd.DataFrame([go(config,deleteSensitiveInfo=True) for _ in range(10)])
results_3.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.640000,0.640201,0.169819,0.151516,0.110470,0.131068,0.201227,0.220901,0.164572,0.183471,...,0.899605,0.889533,0.815699,0.750506,0.713579,0.658613,0.685279,0.783978,0.678311,0.077548
std,0.075396,0.077214,0.095221,0.121701,0.090969,0.115410,0.135328,0.143692,0.117603,0.114115,...,0.208232,0.267588,0.298512,0.363668,0.390712,0.448969,0.467776,0.701931,0.071324,0.051628
min,0.466667,0.465415,0.047619,0.033333,0.013072,0.037879,0.050000,0.053030,0.016667,0.027778,...,0.546875,0.416667,0.416667,0.275862,0.239130,0.136646,0.159420,0.175000,0.572461,0.019668
25%,0.627778,0.626856,0.077778,0.052381,0.041667,0.054167,0.082633,0.104762,0.071429,0.077381,...,0.766790,0.747376,0.541729,0.433190,0.367996,0.367996,0.385237,0.245227,0.637551,0.031835
50%,0.644444,0.650552,0.194697,0.144841,0.103175,0.103175,0.197222,0.214286,0.162500,0.204545,...,0.884476,0.893037,0.881944,0.784056,0.689732,0.567708,0.583333,0.677885,0.673820,0.072540
75%,0.661111,0.663750,0.217262,0.192370,0.128770,0.132738,0.320238,0.255392,0.232472,0.277481,...,1.031937,1.089543,1.016809,0.917411,1.014583,0.907123,0.887477,0.875000,0.703331,0.112750
max,0.777778,0.780000,0.343434,0.434343,0.323232,0.414141,0.414141,0.505051,0.393939,0.333333,...,1.200000,1.259259,1.250000,1.366460,1.275362,1.562500,1.666667,2.428571,0.841866,0.168997


In [26]:
#========= include protected ===========

In [27]:
#Target is Acc 0.79±0.07 Deo 0.04±0.03 
results_4 = pd.DataFrame([go(config,reg=False) for _ in range(10)])
results_4.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.728889,0.727471,0.126863,0.146189,0.161530,0.192843,0.190823,0.171207,0.194250,0.217861,...,0.851300,0.856967,0.832814,0.904732,0.953079,0.937613,0.910004,0.906853,0.649282,0.117220
std,0.086891,0.084796,0.104926,0.115045,0.126948,0.116392,0.129529,0.130195,0.145710,0.154910,...,0.287496,0.317460,0.297345,0.409869,0.522279,0.550560,0.598115,0.616261,0.211711,0.069858
min,0.600000,0.603755,0.017857,0.022222,0.022222,0.022222,0.022222,0.015152,0.022222,0.045455,...,0.538462,0.538462,0.538462,0.538462,0.471154,0.269231,0.269231,0.269231,0.402307,0.027748
25%,0.655556,0.656481,0.060606,0.054067,0.054067,0.087121,0.059118,0.051948,0.080087,0.112879,...,0.614719,0.614719,0.699198,0.738946,0.760331,0.760331,0.610378,0.534139,0.479395,0.068520
50%,0.744444,0.747024,0.100000,0.128788,0.148990,0.222222,0.222222,0.176768,0.165043,0.165043,...,0.858121,0.845868,0.746074,0.780165,0.808489,0.834554,0.755165,0.795892,0.640071,0.092665
75%,0.800000,0.796400,0.164773,0.222222,0.260101,0.276515,0.295860,0.264813,0.305556,0.305556,...,0.892359,0.862945,0.844953,0.846614,0.931677,0.946084,0.913414,0.926482,0.763751,0.157489
max,0.844444,0.842262,0.366071,0.366071,0.357143,0.357143,0.357143,0.357143,0.454545,0.482143,...,1.361111,1.531250,1.531250,1.895833,2.275000,2.275000,2.275000,2.406250,1.095726,0.239933


In [28]:
results_5 = pd.DataFrame([go(config,floor=True, fdiv=binary_binary_renyi2_differentiable) for _ in range(10)])
results_5.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.766667,0.767190,0.131943,0.142852,0.180276,0.179085,0.209605,0.218768,0.218570,0.220871,...,0.649925,0.648572,0.658546,0.644438,0.627995,0.628590,0.664745,0.630412,0.612935,0.145948
std,0.058561,0.057809,0.083890,0.101223,0.149270,0.141557,0.119011,0.129375,0.152783,0.148463,...,0.166882,0.164407,0.189144,0.184407,0.221276,0.250835,0.272579,0.236881,0.155028,0.074127
min,0.688889,0.690711,0.000000,0.000000,0.000000,0.000000,0.102273,0.011364,0.011364,0.011364,...,0.398551,0.318841,0.260870,0.260870,0.212560,0.212560,0.212560,0.239130,0.293606,0.029447
25%,0.733333,0.738124,0.074405,0.074405,0.079140,0.104482,0.145779,0.157576,0.111553,0.095238,...,0.592732,0.569739,0.581695,0.574026,0.518478,0.491546,0.491546,0.491546,0.546815,0.106967
50%,0.755556,0.754447,0.112698,0.106692,0.138889,0.154762,0.181818,0.209596,0.181818,0.202652,...,0.656119,0.672100,0.672100,0.609258,0.601254,0.595862,0.631418,0.631418,0.590425,0.147128
75%,0.794444,0.797553,0.192235,0.214015,0.258681,0.215909,0.220238,0.258681,0.321429,0.351190,...,0.709346,0.773317,0.784849,0.804293,0.718216,0.718216,0.865041,0.813874,0.737426,0.208084
max,0.888889,0.886364,0.270833,0.314286,0.514286,0.514286,0.514286,0.514286,0.514286,0.437500,...,0.929293,0.850000,0.944444,0.876984,0.986607,1.045455,1.127551,0.954082,0.834313,0.245909


In [29]:
#Target is Acc 0.79±0.07 Deo 0.04±0.03 
results_6 = pd.DataFrame([go(config) for _ in range(10)])
results_6.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.642222,0.641931,0.104962,0.190514,0.214403,0.221326,0.237045,0.243160,0.198209,0.203990,...,0.834897,0.863473,0.786912,0.645013,0.674876,0.610688,0.602495,0.544497,0.645086,0.098582
std,0.059674,0.067180,0.069296,0.144374,0.203001,0.198019,0.202231,0.193667,0.187354,0.184802,...,0.296798,0.335294,0.280898,0.208416,0.262526,0.374742,0.388081,0.403480,0.056342,0.060309
min,0.533333,0.530000,0.017857,0.035714,0.000000,0.027273,0.045455,0.045455,0.011905,0.017857,...,0.546875,0.365385,0.265734,0.219231,0.219231,0.175000,0.218750,0.000000,0.595973,0.017118
25%,0.627778,0.620972,0.074026,0.089692,0.074026,0.104302,0.078977,0.111364,0.051948,0.081250,...,0.573718,0.662784,0.644803,0.570513,0.549947,0.350000,0.283826,0.222917,0.602879,0.051069
50%,0.644444,0.651713,0.103571,0.156115,0.175758,0.141234,0.180556,0.195436,0.129167,0.143398,...,0.714773,0.852871,0.799123,0.653216,0.598214,0.540865,0.560000,0.466667,0.626375,0.105280
75%,0.683333,0.683492,0.121528,0.248264,0.250744,0.250744,0.334821,0.274621,0.303188,0.272412,...,1.139247,1.040404,0.973291,0.794444,0.851739,0.782292,0.825000,0.856250,0.677121,0.130235
max,0.711111,0.728745,0.256944,0.535714,0.660714,0.660714,0.660714,0.660714,0.589286,0.589286,...,1.266667,1.531250,1.155555,0.888889,1.066667,1.368421,1.368421,1.250000,0.756874,0.211712


In [11]:
import sys
sys.path.append('../baselines/')
from ferm import go_ferm
from collections import namedtuple
encoded_data, to_protect, encoded_data_test, to_protect_test = read_arrhythmia(dropmissing=True)
encoded_data.drop(encoded_data.columns[0], axis=1, inplace=True)
encoded_data_test.drop(encoded_data_test.columns[0], axis=1, inplace=True)
encoded_data[encoded_data.columns[0]] = encoded_data[encoded_data.columns[0]].astype('category')
encoded_data_test[encoded_data_test.columns[0]] = encoded_data_test[encoded_data_test.columns[0]].astype('category')
train = namedtuple('_', 'data, target')(encoded_data.drop('Target',axis=1).values, encoded_data.Target.values*2-1)
test = namedtuple('_', 'data, target')(encoded_data_test.drop('Target',axis=1).values, encoded_data_test.Target.values*2-1)
acc, deo = go_ferm(train, test, sensible_feature=0)

Different values of the sensible feature 0 : [0, 1]
Grid search for SVM...
Best Estimator: SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Accuracy test: 0.7777777777777778
Accuracy train: 0.9484029484029484
Renyi test 0.12700006365776062
Renyi train 0.24103963375091553
DEO test: 0.16666666666666663
DEO train: 0.12930077691453945



Grid search for our method...
Best Fair Estimator: FERM(C=1, gamma=0.01, kernel='rbf', sensible_feature=0)
Accuracy test: 0.7555555555555555
Accuracy train: 0.9582309582309583
Renyi test 0.0497160479426384
Renyi train 0.2623184025287628
DEO test: 0.25
DEO train: 0.044617092119866864


In [12]:
#I suspect a drop of lines with missing values... 

In [30]:
for x in [results_1, results_2, results_3, results_4, results_5, results_6]:
    dx = x.describe()
    print( "$%.2f {\pm} %.2f $ & $ %.2f {\pm} %.2f$ & " % (dx['accuracy']['mean'], dx['accuracy']['std'],
                                                   dx['deo_0.50']['mean'], dx['deo_0.50']['std'])
             )

$0.73 {\pm} 0.08 $ & $ 0.16 {\pm} 0.11$ & 
$0.77 {\pm} 0.09 $ & $ 0.15 {\pm} 0.07$ & 
$0.64 {\pm} 0.08 $ & $ 0.22 {\pm} 0.14$ & 
$0.73 {\pm} 0.09 $ & $ 0.17 {\pm} 0.13$ & 
$0.77 {\pm} 0.06 $ & $ 0.22 {\pm} 0.13$ & 
$0.64 {\pm} 0.06 $ & $ 0.24 {\pm} 0.19$ & 
